In [1]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from datasets import load_dataset

folder="my-data-chat/" 
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

In [5]:
def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

In [8]:
dataset = load_dataset("b-mc2/sql-create-context", split="train")
len(dataset)

78577

In [10]:
dataset[10]

{'context': 'CREATE TABLE management (temporary_acting VARCHAR)',
 'answer': 'SELECT COUNT(DISTINCT temporary_acting) FROM management',
 'question': 'How many acting statuses are there?'}

In [11]:
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(150))
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
dataset = dataset.train_test_split(test_size=50/150)
dataset_test_valid = dataset['test'].train_test_split(0.5)
print(dataset["train"][45]["messages"])

Map: 100%|██████████| 150/150 [00:00<00:00, 11266.33 examples/s]

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_19948664_2 (index__year_ VARCHAR, countries_sampled VARCHAR)', 'role': 'system'}, {'content': 'How many years were listed under index when there were 157 countries sampled?', 'role': 'user'}, {'content': 'SELECT COUNT(index__year_) FROM table_19948664_2 WHERE countries_sampled = 157', 'role': 'assistant'}]


In [12]:
dataset["train"].to_json(folder + "train.jsonl", orient="records")
dataset_test_valid["train"].to_json(folder + "test.jsonl", orient="records")
dataset_test_valid["test"].to_json(folder + "valid.jsonl", orient="records")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1070.79ba/s]


12516